In [2]:
import pandas as pd
import numpy as np

In [34]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendarUSFH
cal = calendarUSFH()
holidays = cal.holidays(return_name=True, start=pd.to_datetime('20130101'), end=pd.to_datetime('20151231'))
holidays.head(5)

2013-01-01                 New Years Day
2013-01-21    Dr. Martin Luther King Jr.
2013-02-18                Presidents Day
2013-05-27                   MemorialDay
2013-07-04                      July 4th
dtype: object

In [35]:
holidays.to_frame().head(5)

,0
2013-01-01,New Years Day
2013-01-21,Dr. Martin Luther King Jr.
2013-02-18,Presidents Day
2013-05-27,MemorialDay
2013-07-04,July 4th


In [3]:
station = pd.read_csv('data/station.csv', usecols=['id','city'])
station.head(3)

,id,city
0,2,San Jose
1,3,San Jose
2,4,San Jose


In [4]:
def preprocessStation(df):
    df["zip_code"] = df.city.apply(zipCode)
    df.drop(labels=['city'], axis=1, inplace=True)

def zipCode(city):
    if city == 'San Francisco' : return 94107
    elif city == 'San Jose' : return 95113
    elif city == 'Redwood City' : return 94063
    elif city == 'Palo Alto' : return 94301
    elif city == 'Mountain View' : return 94041

In [5]:
preprocessStation(station)

In [6]:
station.head(3)

,id,zip_code
0,2,95113
1,3,95113
2,4,95113


In [7]:
trip = pd.read_csv('data/trip_train.csv',
                   usecols=['duration', 'start_date', 'start_station_id', 'subscription_type'])

In [38]:
def preprocessTrip(df):
    df['start_date'] = df['start_date'].apply(lambda x : pd.to_datetime(x))
    df['time'] = df.start_date.apply(lambda x: 60 * x.hour + x.minute)
    df['year'] = df.start_date.apply(lambda x : x.year)
    df['month'] = df.start_date.apply(lambda x : x.month)
    df['day'] = df.start_date.apply(lambda x : x.day)
    df['dayofweek'] = df.start_date.apply(lambda x : x.isoweekday())
#    df = df.merge(holidays.to_frame(), left_on='start_date', right_on=holidays.to_frame().index, how='outer')
    return df

In [39]:
preprocessTrip(trip)
trip.head(5)

,duration,start_date,start_station_id,subscription_type,time,year,month,day,dayofweek
0,396,2015-08-27 08:36:00,50,Subscriber,516,2015,8,27,4
1,636,2014-07-28 22:06:00,67,Subscriber,1326,2014,7,28,1
2,334,2014-06-09 08:42:00,77,Subscriber,522,2014,6,9,1
3,666,2015-01-26 16:55:00,69,Subscriber,1015,2015,1,26,1
4,318,2015-08-29 15:09:00,67,Subscriber,909,2015,8,29,6


In [10]:
weather = pd.read_csv('data/weather.csv')

In [22]:
def preprocessWeather(df):
    cleanPrecipitation(df)
    cleanEvents(df)
    df = df.fillna(0)
    df['date'] = df.date.apply(lambda x: pd.to_datetime(x).date())
    return df
    
def cleanPrecipitation(df):
    df.dropna(subset=["precipitation_inches"], inplace=True)
    df['precipitation_inches'] = df.precipitation_inches.apply(lambda x:
                                                               0.005 if (x == 'T')
                                                               else float(x))

def cleanEvents(df):
    df['events'] = df.events.apply(lambda x: "Normal" if pd.isnull(x) else x)
    df['events'] = df.events.apply(lambda x: "Rain" if x == "rain" else x)

In [24]:
preprocessWeather(weather).head(5)

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,2013-08-29,74.0,68.0,61.0,61.0,58.0,56.0,93.0,75.0,57.0,...,10.0,10.0,23.0,11.0,28.0,0.0,4.0,Normal,286.0,94107
1,2013-08-30,78.0,69.0,60.0,61.0,58.0,56.0,90.0,70.0,50.0,...,10.0,7.0,29.0,13.0,35.0,0.0,2.0,Normal,291.0,94107
2,2013-08-31,71.0,64.0,57.0,57.0,56.0,54.0,93.0,75.0,57.0,...,10.0,10.0,26.0,15.0,31.0,0.0,4.0,Normal,284.0,94107
3,2013-09-01,74.0,66.0,58.0,60.0,56.0,53.0,87.0,68.0,49.0,...,10.0,10.0,25.0,13.0,29.0,0.0,4.0,Normal,284.0,94107
4,2013-09-02,75.0,69.0,62.0,61.0,60.0,58.0,93.0,77.0,61.0,...,10.0,6.0,23.0,12.0,30.0,0.0,6.0,Normal,277.0,94107


In [ ]:
t